<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_dyson_stability_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_dyson_stability.py

Full training pipeline for DysonSphereAI:

1. Synthetic dataset of 5 Dyson Sphere parameters → 3 stability metrics
2. Physics-informed residuals enforcing structural & thermal equilibrium
3. MLP with LayerNorm & Dropout for robustness
4. MC-Dropout inference for uncertainty quantification
5. Training loop with AdamW, ReduceLROnPlateau, early stopping
6. Visualizations: training curves, scatter plots, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Synthetic Dyson Sphere Dataset
# ------------------------------------------------------------------------------
class DysonDataset(Dataset):
    def __init__(self, n_samples=6000, seed=42):
        np.random.seed(seed)
        # Inputs: orbital_radius (AU), material_density (kg/m³),
        # tensile_strength (Pa), solar_luminosity (L☉), debris_flux (kg/s/m²)
        r  = np.random.uniform(0.5, 5.0,   (n_samples,1)).astype(np.float32)
        ρ  = np.random.uniform(1000, 8000,(n_samples,1)).astype(np.float32)
        σ  = np.random.uniform(1e8, 1e10,(n_samples,1)).astype(np.float32)
        L  = np.random.uniform(0.5, 2.0,   (n_samples,1)).astype(np.float32)
        φ  = np.random.uniform(1e-6, 1e-3,(n_samples,1)).astype(np.float32)

        X_raw = np.hstack([r, ρ, σ, L, φ])

        # Targets (toy physics):
        # structural_margin ≈ σ / r^2
        sm = (σ / r**2)
        # thermal_margin ≈ L / (4πr^2) * emissivity_factor (≈0.7)
        tm = L / (4*np.pi * r**2) * 0.7
        # orbital_drift ≈ φ / ρ
        od = φ / ρ
        Y_raw = np.hstack([sm, tm, od]).astype(np.float32)
        Y_raw += 0.05 * Y_raw.std(0) * np.random.randn(*Y_raw.shape).astype(np.float32)

        # 1D normalization
        self.X_mean, self.X_std = X_raw.mean(0), X_raw.std(0) + 1e-6
        self.Y_mean, self.Y_std = Y_raw.mean(0), Y_raw.std(0) + 1e-6

        self.X = (X_raw - self.X_mean) / self.X_std
        self.Y = (Y_raw - self.Y_mean) / self.Y_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),
            torch.from_numpy(self.Y[idx])
        )

# ------------------------------------------------------------------------------
# 2. Model Definition
# ------------------------------------------------------------------------------
class DysonSphereAI(nn.Module):
    def __init__(self, input_dim=5, hidden_dims=(64,64), output_dim=3, p_drop=0.1):
        super().__init__()
        layers, dim = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics-Informed Residual
# ------------------------------------------------------------------------------
def physics_residual(pred, inp, stats):
    # Denormalize inputs & predictions
    X_den = inp * stats['X_std'] + stats['X_mean']
    Y_den = pred * stats['Y_std'] + stats['Y_mean']
    sm, tm, od = Y_den.t()
    r, ρ, σ, L, φ = X_den.t()

    # Structural: σ/r² ≈ sm
    res1 = nn.MSELoss()(sm, σ / r**2)
    # Thermal: (L/(4πr²))*0.7 ≈ tm
    res2 = nn.MSELoss()(tm, L / (4 * torch.pi * r**2) * 0.7)
    # Drift: φ/ρ ≈ od
    res3 = nn.MSELoss()(od, φ / ρ)

    return res1 + res2 + res3

def total_loss(pred, true, inp, stats, λ=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, inp, stats)
    return mse + λ * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC-Dropout Inference
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(x))
    arr  = torch.stack(preds, dim=0)    # [T, B, 3]
    mean = arr.mean(0)                  # [B, 3]
    std  = arr.std(0)                   # [B, 3]
    return mean, std

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
def train(model, train_ld, val_ld, stats, device,
          lr=1e-3, wd=1e-5, λ=1.0, max_epochs=150, patience=15):
    model.to(device)
    opt   = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min',
                                                factor=0.5, patience=5)
    best_val, wait = float('inf'), 0
    history = {'train':[], 'val':[]}

    for epoch in range(1, max_epochs+1):
        # Training
        model.train()
        t_loss = 0
        for xb, yb in train_ld:
            xb, yb = xb.to(device), yb.to(device)
            pred    = model(xb)
            loss,_,_ = total_loss(pred, yb, xb, stats, λ)
            opt.zero_grad(); loss.backward(); opt.step()
            t_loss += loss.item() * xb.size(0)
        t_loss /= len(train_ld.dataset)

        # Validation
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for xb, yb in val_ld:
                xb, yb = xb.to(device), yb.to(device)
                pred    = model(xb)
                loss,_,_ = total_loss(pred, yb, xb, stats, λ)
                v_loss += loss.item() * xb.size(0)
        v_loss /= len(val_ld.dataset)

        sched.step(v_loss)
        history['train'].append(t_loss)
        history['val'].append(v_loss)
        print(f"Epoch {epoch:03d} Train {t_loss:.4e} Val {v_loss:.4e}")

        if v_loss < best_val - 1e-6:
            best_val, wait = v_loss, 0
            torch.save(model.state_dict(), "best_dyson_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    model.load_state_dict(torch.load("best_dyson_ai.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(hist):
    plt.figure()
    plt.plot(hist['train'], label='Train')
    plt.plot(hist['val'],   label='Val')
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.legend(); plt.title("Training Curve")
    plt.show()

def plot_scatter(y_true, y_pred, name):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m,M],[m,M],'r--')
    plt.xlabel(f"True {name}"); plt.ylabel(f"Pred {name}")
    plt.title(name)
    plt.show()

def plot_uncertainty_heatmap(model, stats_np, device):
    vals = np.linspace(0.5,5.0,100)
    R, S = np.meshgrid(vals, vals)
    grid = np.zeros((R.size,5),dtype=np.float32)
    grid[:,0], grid[:,2] = R.ravel(), S.ravel()
    for i in (1,3,4):
        grid[:,i] = stats_np['X_mean'][i]

    Xn = torch.from_numpy((grid - stats_np['X_mean'])/
                          stats_np['X_std']).float().to(device)
    mean, std = mc_dropout_predict(model, Xn, T=100)
    std0 = std[:,0].cpu().numpy().reshape(R.shape)

    plt.figure(figsize=(6,5))
    plt.pcolormesh(R, S, std0, cmap='inferno', shading='auto')
    plt.colorbar(label='Std(Structural Margin)')
    plt.xlabel('Orbital Radius (AU)'); plt.ylabel('Tensile Strength (Pa)')
    plt.title('Uncertainty Heatmap')
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare dataset & stats
    ds = DysonDataset(6000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, device=device),
        'X_std' : torch.tensor(ds.X_std,  device=device),
        'Y_mean': torch.tensor(ds.Y_mean, device=device),
        'Y_std' : torch.tensor(ds.Y_std,  device=device),
    }
    stats_np = {
        'X_mean': ds.X_mean, 'X_std': ds.X_std,
        'Y_mean': ds.Y_mean, 'Y_std': ds.Y_std,
    }

    # Split & loaders
    n_val = int(0.2 * len(ds))
    tr, va = random_split(ds, [len(ds)-n_val, n_val])
    tr_ld = DataLoader(tr, batch_size=128, shuffle=True)
    va_ld = DataLoader(va, batch_size=256)

    # Model, train, visualize
    model   = DysonSphereAI().to(device)
    history = train(model, tr_ld, va_ld, stats, device)

    plot_history(history)

    X_all = torch.from_numpy(ds.X).float().to(device)
    with torch.no_grad():
        Yp = model(X_all).cpu().numpy()
    Yt = ds.Y
    for i, name in enumerate(['structural_margin','thermal_margin','orbital_drift']):
        plot_scatter(Yt[:,i], Yp[:,i], name)

    plot_uncertainty_heatmap(model, stats_np, device)